<a href="https://colab.research.google.com/github/a-essa/Sentiment-Analysis-and-Satisfaction-Prediction/blob/master/ProjetTripAdvisor_Dataset_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 2.x selected.
2.0.0


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
from google.colab import files
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import os
import io
from tensorflow import keras
from tensorflow.keras import layers
import keras.preprocessing.text
import tensorflow.keras.backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import multilabel_confusion_matrix as mcm
import tensorflow_hub as hub
tf.keras.backend.clear_session()
np.set_printoptions(precision=3, suppress=True)
pd.options.display.max_colwidth = 1000

Using TensorFlow backend.


In [0]:
print(tf.__version__)

2.0.0


In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd 
df=pd.read_csv('/content/gdrive/My Drive/hotel_reviews.csv', sep=';', header=0)

In [0]:
del df['Unnamed: 0']

In [0]:
def create_dataset(dataframe):
  msk = np.random.rand(len(dataframe)) < 0.8

  train = dataframe[msk]

  test = dataframe[~msk]
  #print(train_file_path.head)
  all_data = np.array(dataframe.values.tolist())
  training_set = np.array(train.values.tolist())
  test_set=np.array(test.values.tolist())
  print("Dataset Length: ",len(training_set)+ len(test_set))
  return all_data, training_set, test_set

In [0]:
all_data, training_set, test_set =create_dataset(df)

Dataset Length:  10000


In [0]:
import nltk

nltk.download("stopwords")
nltk.download("punkt")

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
  
def clean_set(set_len):
  stop_words = list(stopwords.words('english'))
  stop_words.remove("not")
  stop_words.remove("no")
  stop_words.remove("isn't")
  stop_words.remove("hasn't")
  stop_words.remove("wasn't")
  stop_words.remove("didn't")
  stop_words.remove("haven't")
  stop_words.remove("don't")
  stop_words.remove("doesn't")
  #stop_words.remove("didn't")
  stop_words.remove("weren't")
  stop_words.remove("shouldn't")
  lemmatizer = WordNetLemmatizer() 
  for elem in set_len:
    word_tokens = word_tokenize(elem[1]) 
    filtered_sentence = [] 
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(lemmatizer.lemmatize(w)) 
    elem[1]=' '.join(filtered_sentence)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
clean_set(all_data)

#IMDB


In [0]:
train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], 
                                  batch_size=-1, as_supervised=True)

train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)

In [0]:
label_train_examples = []
for i in train_examples :
  label_train_examples.append(i.decode("utf-8"))

In [0]:
label_test_examples = []
for i in test_examples :
  label_test_examples.append(i.decode("utf-8"))

In [0]:
train_examples_data = {'label':train_labels ,
                       'text':label_train_examples
                       }

In [0]:
test_examples_data = {'label':test_labels ,
                       'text':label_test_examples
                       }

In [0]:
train_examples_data_tfds = pd.DataFrame(train_examples_data)
test_examples_data_tfds = pd.DataFrame(test_examples_data)

In [0]:
train_examples_data_tfds = np.array(train_examples_data_tfds.values.tolist())
test_examples_data_tfds=np.array(test_examples_data_tfds.values.tolist())

In [0]:
clean_set(train_examples_data_tfds)
clean_set(test_examples_data_tfds)

In [0]:
all_data_tfds = np.append(train_examples_data_tfds, test_examples_data_tfds, axis=0)

#Somme

In [0]:
all_data_tripAdvisor_tfds= np.append(all_data_tfds, all_data, axis=0)

In [0]:
all_data_tripAdvisor_tfds_DataFrame = pd.DataFrame(all_data_tripAdvisor_tfds)

In [0]:
training_set_TripAdvisor_DataFrame = pd.DataFrame(training_set)

In [0]:
test_set_TripAdvisor_DataFrame = pd.DataFrame(test_set)

In [0]:
train_examples_data_IMDB_DataFrame = pd.DataFrame(train_examples_data_tfds)

In [0]:
test_examples_data_IMDB_DataFrame = pd.DataFrame(test_examples_data_tfds)

In [0]:
all_data_tripAdvisor_tfds_DataFrame.to_csv("Total_data_for_tokenizer.csv", sep='\t')

In [0]:
training_set_TripAdvisor_DataFrame.to_csv("training_set_TripAdvisor.csv", sep='\t')

In [0]:
test_set_TripAdvisor_DataFrame.to_csv("test_set_TripAdvisor.csv", sep='\t')

In [0]:
train_examples_data_IMDB_DataFrame.to_csv("training_set_IMDB.csv", sep='\t')

In [0]:
test_examples_data_IMDB_DataFrame.to_csv("test_set_IMDB.csv", sep='\t')

In [0]:
from google.colab import files

In [0]:
files.download('/content/Total_data_for_tokenizer.csv') 

In [0]:
files.download('/content/training_set_TripAdvisor.csv') 

In [0]:
files.download('/content/test_set_TripAdvisor.csv') 

In [0]:
files.download('/content/training_set_IMDB.csv') 

In [0]:
files.download('/content/test_set_IMDB.csv') 